# Introduction

In this tutorial, it is evaluated the HCDF function proposed by Ramoneda et at. [1] on the symbolic domain. Originally, this algorithm was proposed on the audio domain.

The data used for the evaluation is from the Haydn op20 dataset [2]. All the quartet movements scores of the Haydn op 20 are annotated with chords. This dataset is loaded with the mirdata library[3].




>[1] Ramoneda, P., & Bernardes, G. (2020, October). Revisiting Harmonic Change Detection. In Audio Engineering Society Convention 149. Audio Engineering Society.

>[2] López, N. N. (2017). Automatic harmonic analysis of classical string quartets from symbolic score (Doctoral dissertation, Master’s thesis, Universitat Pompeu Fabra).

>[3] Bittner, R. M., Fuentes, M., Rubinstein, D., Jansson, A., Choi, K., & Kell, T. (2019). mirdata: Software for Reproducible Usage of Datasets. In ISMIR (pp. 99-106).

---

Firstly, It is imported a TIVlib [4] in-house version.



>[4] Ramires, António, et al. "TIV. lib: an open-source library for the tonal description of musical audio." arXiv preprint arXiv:2008.11529 (2020).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from astropy.convolution import convolve, Gaussian1DKernel
from scipy.spatial.distance import cosine, euclidean
np.seterr(all='raise')



class TIV:
    weights_symbolic = [2, 11, 17, 16, 19, 7]
    weights = [3, 8, 11.5, 15, 14.5, 7.5]

    def __init__(self, energy, vector):
        self.energy = energy
        self.vector = vector

    @classmethod
    def from_pcp(cls, pcp, symbolic=False):
        if not everything_is_zero(pcp):
            fft = np.fft.rfft(pcp, n=12)
            energy = fft[0]
            vector = fft[1:7]
            if symbolic:
                vector = ((vector / energy) * cls.weights_symbolic)
            else:
                vector = ((vector / energy) * cls.weights)
            return cls(energy, vector)
        else:
            return cls(complex(0), np.array([0, 0, 0, 0, 0, 0]).astype(complex))

    def get_vector(self):
        return np.array(self.vector)

    def dissonance(self):
        return 1 - (np.linalg.norm(self.vector) / np.sqrt(np.sum(np.dot(self.weights, self.weights))))

    def coefficient(self, ii):
        return self.mags()[ii] / self.weights[ii]

    def chromaticity(self):
        return self.mags()[0] / self.weights[0]

    def dyadicity(self):
        return self.mags()[1] / self.weights[1]

    def triadicity(self):
        return self.mags()[2] / self.weights[2]

    def diminished_quality(self):
        return self.mags()[3] / self.weights[3]

    def diatonicity(self):
        return self.mags()[4] / self.weights[4]

    def wholetoneness(self):
        return self.mags()[5] / self.weights[5]

    def mags(self):
        return np.abs(self.vector)

    def plot_tiv(self):
        titles = ["m2/M7", "TT", "M3/m6", "m3/M6", "P4/P5", "M2/m7"]
        tivs_vector = self.vector / self.weights
        i = 1
        for tiv in tivs_vector:
            circle = plt.Circle((0, 0), 1, fill=False)
            plt.subplot(2, 3, i)
            plt.subplots_adjust(hspace=0.4)
            plt.gca().add_patch(circle)
            plt.title(titles[i - 1])
            plt.scatter(tiv.real, tiv.imag)
            plt.xlim((-1.5, 1.5))
            plt.ylim((-1.5, 1.5))
            plt.grid()
            i = i + 1
        plt.show()

    @classmethod
    def euclidean(cls, tiv1, tiv2):
        return np.linalg.norm(tiv1.vector - tiv2.vector)

    @classmethod
    def cosine(cls, tiv1, tiv2):
        a = np.concatenate((tiv1.vector.real, tiv1.vector.imag), axis=0)
        b = np.concatenate((tiv2.vector.real, tiv2.vector.imag), axis=0)
        if everything_is_zero(a) or everything_is_zero(b):
            distance_computed = euclidean(a, b)
        else:
            distance_computed = cosine(a, b)
        return distance_computed

zero_sequence = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
one_sequence = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


def everything_is_zero(vector):
    for element in vector:
        if element != 0:
            return False
    return True

def complex_to_vector(vector):
    ans = []
    for i in range(0, vector.shape[1]):
        row1 = []
        row2 = []
        for j in range(0, vector.shape[0]):
            row1.append(vector[j][i].real)
            row2.append(vector[j][i].imag)
        ans.append(row1)
        ans.append(row2)
    return np.array(ans)

def tonal_interval_space(chroma, symbolic=False):
    centroid_vector = []
    for i in range(0, chroma.shape[1]):
        each_chroma = [chroma[j][i] for j in range(0, chroma.shape[0])]
        # print(each_chroma)
        if everything_is_zero(each_chroma):
            centroid = [0. + 0.j, 0. + 0.j, 0. + 0.j, 0. + 0.j, 0. + 0.j, 0. + 0.j]
        else:
            tonal = TIV.from_pcp(each_chroma, symbolic)
            centroid = tonal.get_vector()
        centroid_vector.append(centroid)
    return complex_to_vector(np.array(centroid_vector))


def gaussian_blur(centroid_vector, sigma):
    centroid_vector = gaussian_filter(centroid_vector, sigma=sigma)
    return centroid_vector


def get_distance(centroids, dist):
    ans = [0]
    if dist == 'euclidean':
        for j in range(1, centroids.shape[1] - 1):
            sum = 0
            for i in range(0, centroids.shape[0]):
                sum += ((centroids[i][j + 1] - centroids[i][j - 1]) ** 2)
            sum = np.math.sqrt(sum)

            ans.append(sum)

    if dist == 'cosine':
        for j in range(1, centroids.shape[1] - 1):
            a = centroids[:, j - 1]
            b = centroids[:, j + 1]
            if everything_is_zero(a) or everything_is_zero(b):
                distance_computed = euclidean(a, b)
            else:
                distance_computed = cosine(a, b)
            ans.append(distance_computed)
    ans.append(0)

    return np.array(ans)


def get_peaks_hcdf(hcdf_function, rate_centroids_second, symbolic=False):
    changes = [0]
    hcdf_changes = []
    last = 0
    for i in range(2, hcdf_function.shape[0] - 1):
        if hcdf_function[i - 1] < hcdf_function[i] and hcdf_function[i + 1] < hcdf_function[i]:
            hcdf_changes.append(hcdf_function[i])
            if not symbolic:
                changes.append(i / rate_centroids_second)
            else:
              changes.append(i)
            last = i
    return np.array(changes), np.array(hcdf_changes)


def harmonic_change(chroma: list, window_size: int=2048, symbolic: bool=False,
                         sigma: int = 5, dist: str = 'euclidean'):
    chroma = np.array(chroma).transpose()
    centroid_vector = tonal_interval_space(chroma, symbolic=symbolic)

    # blur
    centroid_vector_blurred = gaussian_blur(centroid_vector, sigma)

    # harmonic distance and calculate peaks
    harmonic_function = get_distance(centroid_vector_blurred, dist)

    changes, hcdf_changes = get_peaks_hcdf(harmonic_function, window_size, symbolic)

    return changes, hcdf_changes, harmonic_function












Install and import of ot6her required libraries.

In [ ]:
!pip install git+https://github.com/mir-dataset-loaders/mirdata.git@Pedro/haydn_quartets
!pip install mido
!pip uninstal music21
!pip install music21==6.7.1
!pip install unidecode
!pip install mir_eval

  Cloning https://github.com/mir-dataset-loaders/mirdata.git (to revision Pedro/haydn_quartets) to /tmp/pip-req-build-lxmvdpy4
  Running command git clone -q https://github.com/mir-dataset-loaders/mirdata.git /tmp/pip-req-build-lxmvdpy4
  Running command git checkout -b Pedro/haydn_quartets --track origin/Pedro/haydn_quartets
  Switched to a new branch 'Pedro/haydn_quartets'
  Branch 'Pedro/haydn_quartets' set up to track remote branch 'Pedro/haydn_quartets' from 'origin'.
     |████████████████████████████████| 61kB 3.2MB/s 
     |████████████████████████████████| 5.6MB 5.7MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 92kB 10.3MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
  Created wheel for mirdata: filename=mirdata-0.3.2-cp37-none-any.whl size=6398436 sha256=3acf3ca9418149b5cfaca07546876843709cac11ae8b0a8f67b532e6ee9cc879
  Stored in directory: /tmp/pip-ephem-wheel-cache-om2r2r4u/wheels/c9/4d/c4/072127b980b181778a6d

In [ ]:
import music21
import mido
import mirdata
import os
import sys
import mir_eval
import plotly.express as px
import pandas as pd
from mido import MidiFile
import numpy as np
from unidecode import unidecode

Load and validate of haydn op20 with mirdata library

In [ ]:
h20 = mirdata.initialize('haydn_op20')
h20.download()
h20.validate()

INFO: Downloading ['all'] to /root/mir_datasets/haydn_op20
INFO: [all] downloading haydnop20v1.3_annotated.zip
128kB [00:00, 182kB/s]                             
INFO: 
100%|██████████| 24/24 [00:00<00:00, 6613.45it/s]
INFO: Success: the dataset is complete and all files are valid.
INFO: --------------------


({'tracks': {}}, {'tracks': {}})

Example of chord annotation in a random quartet movement.

In [ ]:
h20.choice_track().chords

[{'chord': 'A-major triad', 'time': 0},
 {'chord': 'A-major triad', 'time': 84},
 {'chord': 'E-major triad', 'time': 140},
 {'chord': 'F#-minor triad', 'time': 196},
 {'chord': 'C#-minor triad', 'time': 252},
 {'chord': 'D-major triad', 'time': 308},
 {'chord': 'A-major triad', 'time': 364},
 {'chord': 'E-dominant seventh chord', 'time': 392},
 {'chord': 'A-major triad', 'time': 448},
 {'chord': 'E-major triad', 'time': 532},
 {'chord': 'B-major triad', 'time': 588},
 {'chord': 'C#-minor triad', 'time': 644},
 {'chord': 'B-major triad', 'time': 700},
 {'chord': 'A-major triad', 'time': 756},
 {'chord': 'E-major triad', 'time': 812},
 {'chord': 'F#-minor triad', 'time': 840},
 {'chord': 'B-major triad', 'time': 868},
 {'chord': 'E-major triad', 'time': 896},
 {'chord': 'A-major triad', 'time': 952},
 {'chord': 'C#-minor triad', 'time': 1008},
 {'chord': 'F#-minor triad', 'time': 1064},
 {'chord': 'E-major triad', 'time': 1120},
 {'chord': 'D-major triad', 'time': 1176},
 {'chord': 'C#-m

Import utility functions for dealing with piano rolls

In [ ]:


#######
# Pianorolls dims are  :   TIME  *  PITCH


class Read_midi(object):
    def __init__(self, song_path, quantization):
        ## Metadata
        self.__song_path = song_path
        self.__quantization = quantization

        ## Pianoroll
        self.__T_pr = None

        ## Private misc
        self.__num_ticks = None
        self.__T_file = None

    @property
    def quantization(self):
        return self.__quantization

    @property
    def T_pr(self):
        return self.__T_pr

    @property
    def T_file(self):
        return self.__T_file

    def get_total_num_tick(self):
        # Midi length should be written in a meta message at the beginning of the file,
        # but in many cases, lazy motherfuckers didn't write it...

        # Read a midi file and return a dictionnary {track_name : pianoroll}
        mid = MidiFile(self.__song_path)

        # Parse track by track
        num_ticks = 0
        for i, track in enumerate(mid.tracks):
            tick_counter = 0
            for message in track:
                # Note on
                time = float(message.time)
                tick_counter += time
            num_ticks = max(num_ticks, tick_counter)
        self.__num_ticks = num_ticks

    def get_pitch_range(self):
        mid = MidiFile(self.__song_path)
        min_pitch = 200
        max_pitch = 0
        for i, track in enumerate(mid.tracks):
            for message in track:
                if message.type in ['note_on', 'note_off']:
                    pitch = message.note
                    if pitch > max_pitch:
                        max_pitch = pitch
                    if pitch < min_pitch:
                        min_pitch = pitch
        return min_pitch, max_pitch

    def get_time_file(self):
        # Get the time dimension for a pianoroll given a certain quantization
        mid = MidiFile(self.__song_path)
        # Tick per beat
        ticks_per_beat = mid.ticks_per_beat
        # Total number of ticks
        self.get_total_num_tick()
        # Dimensions of the pianoroll for each track
        self.__T_file = int((self.__num_ticks / ticks_per_beat) * self.__quantization)
        return self.__T_file

    def read_file(self):
        # Read the midi file and return a dictionnary {track_name : pianoroll}
        mid = MidiFile(self.__song_path)
        # Tick per beat
        ticks_per_beat = mid.ticks_per_beat

        # Get total time
        self.get_time_file()
        T_pr = self.__T_file
        # Pitch dimension
        N_pr = 128
        pianoroll = {}

        def add_note_to_pr(note_off, notes_on, pr):
            pitch_off, _, time_off = note_off
            # Note off : search for the note in the list of note on,
            # get the start and end time
            # write it in th pr
            match_list = [(ind, item) for (ind, item) in enumerate(notes_on) if item[0] == pitch_off]
            if len(match_list) == 0:
                print("Try to note off a note that has never been turned on")
                # Do nothing
                return

            # Add note to the pr
            pitch, velocity, time_on = match_list[0][1]
            pr[time_on:time_off, pitch] = velocity
            # Remove the note from notes_on
            ind_match = match_list[0][0]
            del notes_on[ind_match]
            return

        # Parse track by track
        counter_unnamed_track = 0
        for i, track in enumerate(mid.tracks):
            # Instanciate the pianoroll
            pr = np.zeros([T_pr, N_pr])
            time_counter = 0
            notes_on = []
            for message in track:

                ##########################################
                ##########################################
                ##########################################
                # TODO : keep track of tempo information
                # import re
                # if re.search("tempo", message.type):
                #     import pdb; pdb.set_trace()
                ##########################################
                ##########################################
                ##########################################


                # print message
                # Time. Must be incremented, whether it is a note on/off or not
                time = float(message.time)
                time_counter += time / ticks_per_beat * self.__quantization
                # Time in pr (mapping)
                time_pr = int(round(time_counter))
                # Note on
                if message.type == 'note_on':
                    # Get pitch
                    pitch = message.note
                    # Get velocity
                    velocity = message.velocity
                    if velocity > 0:
                        notes_on.append((pitch, velocity, time_pr))
                    elif velocity == 0:
                        add_note_to_pr((pitch, velocity, time_pr), notes_on, pr)
                # Note off
                elif message.type == 'note_off':
                    pitch = message.note
                    velocity = message.velocity
                    add_note_to_pr((pitch, velocity, time_pr), notes_on, pr)

            # We deal with discrete values ranged between 0 and 127
            #     -> convert to int
            pr = pr.astype(np.int16)
            if np.sum(np.sum(pr)) > 0:
                name = unidecode(track.name)
                name = name.rstrip('\x00')
                if name == u'':
                    name = 'unnamed' + str(counter_unnamed_track)
                    counter_unnamed_track += 1
                if name in pianoroll.keys():
                    # Take max of the to pianorolls
                    pianoroll[name] = np.maximum(pr, pianoroll[name])
                else:
                    pianoroll[name] = pr
        return pianoroll

Example of hcdf across one quartet movement

In [ ]:
choice = h20.load_tracks()['0']
midi_matrixes = Read_midi(choice.midi_path, 28).read_file()



In [ ]:
for k, t in midi_matrixes.items():
  print(t.shape)

(12208, 128)
(12208, 128)
(12208, 128)
(12208, 128)


In [ ]:
mat = list(midi_matrixes.values())
midi_quartet = mat[0] + mat[1] + mat[2] + mat[3]
midi_quartet.shape

(12208, 128)

In [ ]:

np.set_printoptions(threshold=sys.maxsize)

In [ ]:
def midi2chroma(midi_vector):
    chroma_vector = np.zeros((midi_vector.shape[0], 12))
    for ii, midi_frame in enumerate(midi_vector):
        for jj, element in enumerate(midi_frame):
            chroma_vector[ii][jj % 12] += element
    return chroma_vector

chroma_quartets = midi2chroma(midi_quartet)

In [ ]:
changes, hcdf_changes, harmonic_function = harmonic_change(chroma=chroma_quartets, symbolic=True,
                         sigma=28, dist='euclidean')
changes

array([    0,    23,   114,   173,   269,   333,   394,   465,   566,
         624,   671,   710,   740,   796,   864,   954,  1015,  1069,
        1136,  1205,  1300,  1354,  1415,  1510,  1578,  1605,  1688,
        1743,  1792,  1858,  1930,  1997,  2050,  2111,  2177,  2246,
        2310,  2421,  2477,  2575,  2639,  2716,  2768,  2897,  2966,
        3077,  3153,  3223,  3348,  3417,  3468,  3530,  3584,  3630,
        3694,  3772,  3848,  3916,  3997,  4054,  4185,  4250,  4309,
        4368,  4431,  4495,  4557,  4618,  4724,  4785,  4840,  4885,
        4930,  4989,  5076,  5135,  5218,  5301,  5385,  5493,  5567,
        5623,  5692,  5745,  5793,  5883,  5995,  6102,  6201,  6261,
        6377,  6444,  6493,  6556,  6634,  6757,  6836,  6889,  6950,
        7112,  7189,  7231,  7282,  7426,  7470,  7511,  7601,  7664,
        7715,  7783,  7840,  7984,  8045,  8078,  8110,  8144,  8187,
        8237,  8333,  8396,  8453,  8532,  8597,  8691,  8748,  8806,
        8856,  8907,

In [ ]:
changes_ground_truth = np.array([c['time'] for c in choice.chords])
changes_ground_truth

array([    0,   364,   392,   644,   672,   728,   784,   896,   952,
        1064,  1120,  1176,  1232,  1344,  1400,  1456,  1512,  1540,
        1568,  1680,  1736,  1848,  2072,  2114,  2128,  2240,  2268,
        2296,  2338,  2352,  2408,  2436,  2632,  2800,  2856,  2968,
        2996,  3024,  3192,  3220,  3360,  3388,  3416,  3444,  3458,
        3472,  3528,  3584,  3696,  3808,  3920,  4004,  4018,  4032,
        4130,  4144,  4480,  4508,  4816,  5208,  5320,  5488,  5544,
        5656,  5768,  5880,  5992,  6104,  6216,  6328,  6384,  6440,
        6496,  6552,  6664,  6692,  6720,  6888,  6916,  6944,  7112,
        7252,  7280,  7392,  7448,  7504,  7896,  8064,  8568,  8624,
        8736,  8792,  8848,  8904,  8932,  8960,  8988,  9016,  9044,
        9072,  9100,  9128,  9184,  9464,  9520,  9632,  9744,  9856,
       10024, 10136, 10164, 10192, 10248, 10304, 10360, 10416, 10472,
       10528, 10556, 10584, 10612, 10738, 10752, 10892, 10920, 10948,
       11088, 11116,

In [ ]:
f_measure, precision, recall = mir_eval.onset.f_measure(changes_ground_truth, changes, window=31.218) #same window than Harte
f_measure, precision, recall

(0.5850746268656717, 0.547486033519553, 0.6282051282051282)

# HCDF evaluation across the haydn op20 dataset

In [ ]:

def evaluate_hcdf_across_haydn_op20(sigma=30, distance='euclidean'):
    f_measure_results = []
    precision_results = []
    recall_results = []
    print("evaluate_hcdf_across_haydn_op20", sigma, distance)
    for k, t in h20.load_tracks().items():
        midi_matrixes = Read_midi(t.midi_path, 28).read_file()
        mat = list(midi_matrixes.values())
        midi_quartet = mat[0] + mat[1] + mat[2] + mat[3]
        chroma_quartets = midi2chroma(midi_quartet)
        
        changes, hcdf_changes, harmonic_function = harmonic_change(chroma=chroma_quartets, symbolic=True,
                            sigma=sigma, dist=distance)

        changes_ground_truth = np.array([c['time'] for c in t.chords])

        f_measure, precision, recall = mir_eval.onset.f_measure(changes_ground_truth, changes, window=31.218) #same window than Harte
        # print(t.title, f_measure, precision, recall)
        f_measure_results.append(f_measure)
        precision_results.append(precision)
        recall_results.append(recall)
    return np.mean(np.array(f_measure_results)), \
           np.mean(np.array(precision_results)), \
           np.mean(np.array(recall_results))

evaluate_hcdf_across_haydn_op20()

evaluate_hcdf_across_haydn_op20 30 euclidean


(0.676493830808618, 0.835155380355339, 0.5907711748661402)

In [ ]:
results_euclidean = {
    sigma: evaluate_hcdf_across_haydn_op20(sigma=sigma, distance='euclidean') 
    for sigma in range(1, 52, 5)
}

evaluate_hcdf_across_haydn_op20 1 euclidean
evaluate_hcdf_across_haydn_op20 6 euclidean
evaluate_hcdf_across_haydn_op20 11 euclidean
evaluate_hcdf_across_haydn_op20 16 euclidean
evaluate_hcdf_across_haydn_op20 21 euclidean
evaluate_hcdf_across_haydn_op20 26 euclidean
evaluate_hcdf_across_haydn_op20 31 euclidean
evaluate_hcdf_across_haydn_op20 36 euclidean
evaluate_hcdf_across_haydn_op20 41 euclidean
evaluate_hcdf_across_haydn_op20 46 euclidean
evaluate_hcdf_across_haydn_op20 51 euclidean


In [ ]:

def tune_sigma_plot(evaluation_result):
    sigma_list = []; type_metric = []; metrics = []
    for s, v in evaluation_result.items():
        f, p, r = v
        # f measure
        sigma_list.append(s)
        type_metric.append("F_score")
        metrics.append(f)
        # Precision
        sigma_list.append(s)
        type_metric.append("Precision")
        metrics.append(p)
        # Recall
        sigma_list.append(s)
        type_metric.append("Recall")
        metrics.append(r)
    df_dict = {
        "sigma": sigma_list,
        "metric": type_metric,
        "value": metrics
    }

    df = pd.DataFrame(df_dict)
    fig = px.line(df, x="sigma", y="value", color="metric", render_mode="svg")
    fig.show()


Tuning sigma gaussian hyperparameter for HCDF with euclidean distance.

In [ ]:
tune_sigma_plot(results_euclidean)

Results are better segmenting the chord boundaries that the current approaches for chord recognition in symbolic domain. With a sigma=20 all the metrics computed across the Haydn op 20 dataset are greater than 70%.
Due to chord analysis subjectivity the results are enough good for using this function to segment harmonically symbolic data.

In [29]:
results_cosine = {
    sigma: evaluate_hcdf_across_haydn_op20(sigma=sigma, distance='cosine') 
    for sigma in range(1, 52, 5)
}
tune_sigma_plot(results_cosine)

evaluate_hcdf_across_haydn_op20 1 cosine
evaluate_hcdf_across_haydn_op20 6 cosine
evaluate_hcdf_across_haydn_op20 11 cosine
evaluate_hcdf_across_haydn_op20 16 cosine
evaluate_hcdf_across_haydn_op20 21 cosine
evaluate_hcdf_across_haydn_op20 26 cosine
evaluate_hcdf_across_haydn_op20 31 cosine
evaluate_hcdf_across_haydn_op20 36 cosine
evaluate_hcdf_across_haydn_op20 41 cosine
evaluate_hcdf_across_haydn_op20 46 cosine
evaluate_hcdf_across_haydn_op20 51 cosine


The performance of HCDF with the cosine distance is a bit worse than using the euclidean distance.